# Importando bibliotecas e o dataset:

In [1]:
!pip install chardet

# Importar bibliotecas
import pandas as pd
import chardet
import seaborn as sns
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Definir caminho para o arquivo
file_path = '/content/drive/MyDrive/spotify-2023.csv'

# Detectar codificação
with open(file_path, 'rb') as f:
    result = chardet.detect(f.read(10000))
encoding_detectado = result['encoding']
print(f'Encoding detectado: {encoding_detectado}')

# Ler dataset com codificação correta e separador explícito
df = pd.read_csv(file_path, encoding=encoding_detectado, sep=',')

Mounted at /content/drive
Encoding detectado: ISO-8859-1


# Probabilidade de uma nova música aparecer em playlists:

In [7]:
from scipy.stats import binom

colunas_playlists = [
    'in_spotify_playlists', 'in_apple_playlists',
    'in_deezer_playlists'
]

df[colunas_playlists] = df[colunas_playlists].apply(pd.to_numeric, errors='coerce').fillna(0)
df['em_qualquer_playlist'] = (df[colunas_playlists].sum(axis=1) > 0).astype(int)
df = df.dropna(subset=['artist(s)_name'])

prob_playlist_por_artista = df.groupby('artist(s)_name')['em_qualquer_playlist'].mean()

def prob_playlist_artista(nome_artista):
    if nome_artista in prob_playlist_por_artista:
        return prob_playlist_por_artista[nome_artista]
    else:
        return None

def simular_playlists(nome_artista, n_musicas=100):
    p_playlist = prob_playlist_artista(nome_artista)
    if p_playlist is None:
        return f"Artista '{nome_artista}' não encontrado no dataset."

    playlists_simuladas = binom.rvs(n=1, p=p_playlist, size=n_musicas)
    prob_simulada = playlists_simuladas.mean()

    return {
        'artista': nome_artista,
        'prob_playlist_historica': p_playlist * 100,
        'musicas_simuladas': n_musicas,
        'musicas_em_playlist': playlists_simuladas.sum(),
        'prob_playlist_simulada': prob_simulada * 100
    }

resultado_playlist = simular_playlists('Taylor Swift', n_musicas=100)

if isinstance(resultado_playlist, dict):
    print(f"Artista: {resultado_playlist['artista']}")
    print(f"Probabilidade histórica de estar em playlist: {resultado_playlist['prob_playlist_historica']:.2f}%")
    print(f"Músicas em playlist em {resultado_playlist['musicas_simuladas']} lançamentos: {resultado_playlist['musicas_em_playlist']}")
    print(f"Probabilidade simulada de estar em playlist: {resultado_playlist['prob_playlist_simulada']:.2f}%")
else:
    print(resultado_playlist)




Artista: Taylor Swift
Probabilidade histórica de estar em playlist: 100.00%
Músicas em playlist em 100 lançamentos: 100
Probabilidade simulada de estar em playlist: 100.00%


# Probabilidade de uma nova música aparecer em um charts

In [6]:
import pandas as pd
from scipy.stats import binom

colunas_charts = [
    'in_spotify_charts', 'in_apple_charts',
    'in_deezer_charts', 'in_shazam_charts'
]

df[colunas_charts] = df[colunas_charts].apply(pd.to_numeric, errors='coerce').fillna(0)
df['em_qualquer_chart'] = (df[colunas_charts].sum(axis=1) > 0).astype(int)
df = df.dropna(subset=['artist(s)_name'])

prob_chart_por_artista = df.groupby('artist(s)_name')['em_qualquer_chart'].mean()

def prob_chart_artista(nome_artista):
    if nome_artista in prob_chart_por_artista:
        return prob_chart_por_artista[nome_artista]
    else:
        return None

def simular_charts(nome_artista, n_musicas=100):
    p_chart = prob_chart_artista(nome_artista)
    if p_chart is None:
        return f"Artista '{nome_artista}' não encontrado no dataset."

    charts_simulados = binom.rvs(n=1, p=p_chart, size=n_musicas)
    prob_simulada = charts_simulados.mean()

    return {
        'artista': nome_artista,
        'prob_chart_historica': p_chart * 100,
        'musicas_simuladas': n_musicas,
        'musicas_em_chart': charts_simulados.sum(),
        'prob_chart_simulada': prob_simulada * 100
    }

resultado_chart = simular_charts('Taylor Swift', n_musicas=100)

if isinstance(resultado_chart, dict):
    print(f"Artista: {resultado_chart['artista']}")
    print(f"Probabilidade histórica de estar em chart: {resultado_chart['prob_chart_historica']:.2f}%")
    print(f"Músicas em chart em {resultado_chart['musicas_simuladas']} lançamentos: {resultado_chart['musicas_em_chart']}")
    print(f"Probabilidade simulada de estar em chart: {resultado_chart['prob_chart_simulada']:.2f}%")
else:
    print(resultado_chart)


Artista: Taylor Swift
Probabilidade histórica de estar em chart: 97.06%
Músicas em chart em 100 lançamentos: 98
Probabilidade simulada de estar em chart: 98.00%


# Probabilidade de uma nova música se tornar um hit

In [5]:
import pandas as pd
from scipy.stats import binom

df['streams'] = pd.to_numeric(df['streams'], errors='coerce')
df['hit'] = (df['streams'] >= 100_000_000).astype(int)
df = df.dropna(subset=['artist(s)_name'])

prob_hit_por_artista = df.groupby('artist(s)_name')['hit'].mean()

def prob_hit_artista(nome_artista):
    if nome_artista in prob_hit_por_artista:
        return prob_hit_por_artista[nome_artista]
    else:
        return None

def simular_hits(nome_artista, n_musicas=100):
    p_hit = prob_hit_artista(nome_artista)
    if p_hit is None:
        return f"Artista '{nome_artista}' não encontrado no dataset."

    hits_simulados = binom.rvs(n=1, p=p_hit, size=n_musicas)
    prob_simulada = hits_simulados.mean()

    return {
        'artista': nome_artista,
        'prob_hit_historica': p_hit * 100,
        'musicas_simuladas': n_musicas,
        'hits_esperados': hits_simulados.sum(),
        'prob_hit_simulada': prob_simulada * 100
    }

resultado = simular_hits('Taylor Swift', n_musicas=100)

if isinstance(resultado, dict):
    print(f"Artista: {resultado['artista']}")
    print(f"Probabilidade histórica de hit: {resultado['prob_hit_historica']:.2f}%")
    print(f"Hits esperados em {resultado['musicas_simuladas']} músicas novas: {resultado['hits_esperados']}")
    print(f"Probabilidade simulada de hit: {resultado['prob_hit_simulada']:.2f}%")
else:
    print(resultado)



Artista: Taylor Swift
Probabilidade histórica de hit: 82.35%
Hits esperados em 100 músicas novas: 81
Probabilidade simulada de hit: 81.00%
